In [2]:
import os
import threading
import time
import subprocess
import gradio as gr
from langchain.agents import Tool
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent
from functions import get_weather_info, get_forecast

In [3]:
api_key = "c6dfc4d92a8f972d237ef696ec87b37a"

In [4]:
#Initialize Ollama service
OLLAMA = os.path.expanduser("~/ollama")

if not os.path.exists(OLLAMA):
    subprocess.run("curl -L https://ollama.com/download/ollama-linux-amd64 -o ~/ollama", shell=True)
    os.chmod(OLLAMA, 0o755)

def ollama_service_thread():
    subprocess.run("~/ollama serve", shell=True)

OLLAMA_SERVICE_THREAD = threading.Thread(target=ollama_service_thread)
OLLAMA_SERVICE_THREAD.start()

print("Giving ollama serve a moment")
time.sleep(10)


Giving ollama serve a moment


Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [5]:
# Specify the model you want to use
model = "phi3:3.8b-mini-128k-instruct-q4_0"
subprocess.run(f"~/ollama pull {model}", shell=True)

# Initialize the Ollama model
llm = Ollama(model=model, base_url="http://localhost:11434")


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest 
pulling 3e38718d00bb... 100% ▕████████████████▏ 2.2 GB                         
pulling fa8235e5b48f... 100% ▕████████████████▏ 1.1 KB                         
pulling 542b217f179c... 100% ▕████████████████▏  148 B                         
pulling 8dde1baf1db0... 100% ▕████████████████▏   78 B                         
pulling ed7ab7698fdd... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [15]:
# Weather Tool
weather_tool = Tool(
    name="WeatherLookup",
    func=lambda city: get_weather_info(city),
    # func=get_weather_info,
    description="Useful to get the current weather (Today) information for a city. It includes information on temperature, pressure, humidity, wind, clouds and rain."
)

# Forecast Tool
forecast_tool = Tool(
    name="ForecastLookup",
    func=lambda city: get_forecast(city),
    # func=get_forecast,
    description="Useful to get the weather forecast for the next day (Tomorrow) for a city. It includes information on temperature, pressure, humidity, wind, clouds and rain."
)

# Tools (Include both Weather and Forecast Tools)
tools = [weather_tool, forecast_tool]

In [16]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True, 
    handle_parsing_errors=True , # This is crucial,
    # return_intermediate_steps=True
)

In [17]:
try:
    result = agent.run("What's the Current weather in Kalyani?")
    for step in result["intermediate_steps"]:
        print(step)
except Exception as e: 
    print(f"An error occurred: {e}")



> Entering new AgentExecutor chain...
 To find out the current weather conditions of Kalyani, we need to use WeatherLookup action. 

Action: WeatherLookup(Kalyani)
Action Input: None (just specifying city name as input).
Observation: WeatherLookup(Kalyani) is not a valid tool, try one of [WeatherLookup, ForecastLookup].
Thought: I apologize for the confusion earlier. Let's use the correct action to find out the current weather conditions in Kalyani using an alternative method or reliable online resource if necessary.

Action: None (I will provide information based on general knowledge and available data as of my last update).
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:


KeyboardInterrupt

